# Bangladesh Tax Calculator Agent with LangGraph

An intelligent conversational agent that helps calculate:
- Income tax based on Bangladesh tax laws (2025-2026)
- Investment rebates (15% of eligible investments)
- Surcharges (net worth and environmental)
- Tax information and regulations

**Built with:** LangGraph + Google Gemini 2.0

In [ ]:
# Install required packages (uncomment if needed)
# !pip install langgraph langchain-google-genai langchain-core python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Literal
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
print("✓ Imports successful")

## Tool 1: Income Tax Calculator

Calculates income tax using Bangladesh progressive tax slabs (2025-2026).

In [2]:
@tool
def calculate_income_tax(annual_income: float, taxpayer_category: str = "general", location: str = "dhaka_city") -> dict:
    """Calculate income tax based on Bangladesh tax law (2025-2026).
    
    Args:
        annual_income: Total annual income in BDT
        taxpayer_category: 'general', 'female', 'senior', 'disabled', 'freedom_fighter'
        location: 'dhaka_city', 'chittagong_city', 'other_city', 'outside_city'
    """
    thresholds = {'general': 350000, 'female': 400000, 'senior': 400000, 'disabled': 475000, 'freedom_fighter': 500000}
    min_tax = {'dhaka_city': 5000, 'chittagong_city': 5000, 'other_city': 4000, 'outside_city': 3000}
    
    tax_free_limit = thresholds.get(taxpayer_category, 350000)
    taxable_income = max(0, annual_income - tax_free_limit)
    
    tax = 0
    remaining = taxable_income
    breakdown = []
    
    slabs = [(100000, 0.05, "First"), (400000, 0.10, "Next"), (500000, 0.15, "Next"), (2000000, 0.20, "Next"), (float('inf'), 0.25, "Remaining")]
    
    for limit, rate, label in slabs:
        if remaining > 0:
            slab_amount = min(remaining, limit)
            slab_tax = slab_amount * rate
            tax += slab_tax
            breakdown.append(f"{label} BDT {slab_amount:,.0f} @ {rate*100}% = BDT {slab_tax:,.0f}")
            remaining -= slab_amount
    
    minimum_tax = min_tax.get(location, 3000)
    final_tax = max(tax, minimum_tax)
    
    return {
        "annual_income": annual_income, "tax_free_limit": tax_free_limit, "taxable_income": taxable_income,
        "tax_before_minimum": tax, "minimum_tax": minimum_tax, "tax_payable": final_tax,
        "breakdown": breakdown, "effective_rate": (final_tax / annual_income * 100) if annual_income > 0 else 0
    }

print("✓ Tax calculator tool defined")

✓ Tax calculator tool defined


## Tool 2: Investment Rebate Calculator

In [3]:
@tool
def calculate_investment_rebate(taxable_income: float, investment_amount: float) -> dict:
    """Calculate 15% investment rebate on eligible investments."""
    admissible_amount = min(investment_amount, taxable_income * 0.20, 10000000)
    rebate = admissible_amount * 0.15
    
    return {
        "investment_amount": investment_amount, "taxable_income": taxable_income,
        "20_percent_of_income": taxable_income * 0.20, "admissible_amount": admissible_amount,
        "rebate_rate": "15%", "rebate_amount": rebate,
        "eligible_investments": ["Life insurance", "Provident fund", "Govt securities", "Stocks", "Donations"]
    }

print("✓ Rebate calculator tool defined")

✓ Rebate calculator tool defined


## Tool 3: Surcharge Calculator

In [4]:
@tool
def calculate_surcharge(base_tax: float, net_worth: float = 0, vehicle_engine_cc: int = 0) -> dict:
    """Calculate net worth and environmental surcharges."""
    nw_surcharge, nw_rate = 0, 0
    
    if net_worth > 500000000: nw_rate, nw_surcharge = 0.35, base_tax * 0.35
    elif net_worth > 200000000: nw_rate, nw_surcharge = 0.25, base_tax * 0.25
    elif net_worth > 100000000: nw_rate, nw_surcharge = 0.20, base_tax * 0.20
    elif net_worth > 40000000: nw_rate, nw_surcharge = 0.10, base_tax * 0.10
    
    env_map = {3500: 350000, 3000: 200000, 2500: 150000, 2000: 75000, 1500: 50000, 0: 25000}
    env_surcharge = next((v for k, v in env_map.items() if vehicle_engine_cc > k), 0)
    
    return {
        "base_tax": base_tax, "net_worth": net_worth, "net_worth_surcharge_rate": f"{nw_rate * 100}%",
        "net_worth_surcharge": nw_surcharge, "vehicle_engine_cc": vehicle_engine_cc,
        "environmental_surcharge": env_surcharge, "total_surcharge": nw_surcharge + env_surcharge,
        "total_tax_with_surcharge": base_tax + nw_surcharge + env_surcharge
    }

print("✓ Surcharge calculator tool defined")

✓ Surcharge calculator tool defined


## Tool 4: Tax Information Provider

In [5]:
@tool
def get_tax_info(query: str) -> str:
    """Get information about Bangladesh tax regulations."""
    q = query.lower()
    info = {
        "rate": "Tax Rates: 0% (350K), 5% (100K), 10% (400K), 15% (500K), 20% (2M), 25% (above). Special: Female/Senior 400K, Disabled 475K tax-free",
        "exemption": "Exemptions: Agri income (5M max), International awards, Employer insurance, Medical expenses",
        "deadline": "Deadlines: Individuals - Nov 30, Companies - 15th after 6 months from year-end",
        "rebate": "Rebate: 15% of min(investment, 20% income, 1 crore). Eligible: Insurance, PF, Govt securities, Stocks",
        "minimum": "Minimum Tax: Dhaka/Ctg 5K, Other cities 4K, Outside 3K"
    }
    
    for key, val in info.items():
        if key in q: return val
    
    return "BD Tax (2025-26): 350K tax-free, 0-25% rates, 15% rebate, 3-5K minimum. Ask: rates, exemptions, deadline, rebate, minimum"

print("✓ Tax info tool defined")

✓ Tax info tool defined


## Build the LangGraph Agent

In [6]:
# Setup
tools = [calculate_income_tax, calculate_investment_rebate, calculate_surcharge, get_tax_info]
llm = ChatGoogleGenerativeAI(api_key=api_key, model="gemini-2.0-flash-exp", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# State
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# Nodes
def agent_node(state: AgentState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

def should_continue(state: AgentState) -> Literal["tools", END]:
    last = state["messages"][-1]
    return "tools" if hasattr(last, "tool_calls") and last.tool_calls else END

# Build graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")
app = workflow.compile()

print("✓ Tax Calculator Agent Ready!")

✓ Tax Calculator Agent Ready!


## Chat Helper Function

In [7]:
def chat(message: str, conversation_history: list = None):
    """Chat with the tax agent."""
    if conversation_history is None:
        conversation_history = []
    
    if not conversation_history:
        sys = HumanMessage(content="You are a Bangladesh tax expert (2025-26). Use tools for calculations. Explain clearly.")
        conversation_history.append(sys)
    
    conversation_history.append(HumanMessage(content=message))
    result = app.invoke({"messages": conversation_history})
    response = result["messages"][-1].content
    
    return response, result["messages"]

print("✓ Chat function ready")

✓ Chat function ready


---
# Examples

## Example 1: Basic Tax Calculation

In [16]:
conv = []
response, conv = chat("I earn BDT 165k  per month , and get a 165k bonus. . How much tax? I live in Dhaka. and how much should I pay each month ? if I get the max rebate  ", conv)
print(response)

Okay, let's break down your tax situation.

**1. Calculate your total annual income:**

*   Your monthly income is BDT 165,000.
*   Your annual income from salary is BDT 165,000 \* 12 = BDT 1,980,000.
*   You also receive a bonus of BDT 165,000.
*   Your total annual income is BDT 1,980,000 + BDT 165,000 = BDT 2,145,000.

**2. Calculate your income tax:**

*   Based on an annual income of BDT 2,145,000 and living in Dhaka, your tax before rebate is BDT 279,000.

**3. Calculate the maximum investment rebate:**

*   You can get a rebate of 15% on eligible investments, up to a maximum of 20% of your taxable income.
*   20% of your income is BDT 359,000.
*   The maximum rebate you can get is BDT 53,850.

**4. Calculate your tax after the rebate:**

*   Your tax payable after the rebate is BDT 279,000 - BDT 53,850 = BDT 225,150.

**5. Calculate your monthly tax liability:**

*   To find out how much you should pay each month, divide your total tax payable by 12: BDT 225,150 / 12 = BDT 18,76

## Example 2: With Investment Rebate

In [13]:
response, conv = chat("I invested BDT 300,000 in life insurance. What rebate can I get?", conv)
print(response)

You invested BDT 300,000 in life insurance. Based on a taxable income of BDT 1,150,000, the maximum investment amount eligible for rebate is BDT 230,000 (20% of your taxable income). Therefore, your rebate will be calculated on BDT 230,000. At a 15% rebate rate, your investment rebate is BDT 34,500.


## Example 3: Final Tax After Rebate

In [ ]:
response, conv = chat("What's my final tax after applying the rebate?", conv)
print(response)

## Example 4: Female Taxpayer

In [12]:
conv2 = []
response, conv2 = chat("I'm a female earning BDT 800,000 in Chittagong. Calculate my tax.", conv2)
print(response)

Okay, I can help you with that. As a female taxpayer with an annual income of BDT 800,000 in Chittagong city, your tax is calculated as follows:

*   **Tax-free limit:** BDT 400,000
*   **Taxable income:** BDT 400,000 (BDT 800,000 - BDT 400,000)
*   **Tax calculation:**
    *   First BDT 100,000 @ 5% = BDT 5,000
    *   Next BDT 300,000 @ 10% = BDT 30,000
*   **Total tax payable:** BDT 35,000

So, your total income tax payable is BDT 35,000. The effective tax rate is 4.375%.


## Example 5: High Net Worth with Surcharges

In [ ]:
conv3 = []
msg = "Income: BDT 5M, Net worth: BDT 12 crore, Car: 2500cc. Calculate total tax with surcharges."
response, conv3 = chat(msg, conv3)
print(response)

## Example 6: Tax Information Query

In [ ]:
conv4 = []
response, conv4 = chat("What are the tax exemptions available?", conv4)
print(response)

---
## Interactive Chat Mode

Run this cell for a conversational interface:

In [ ]:
def interactive_chat():
    """Interactive chat. Type 'quit' to exit."""
    print("="*60)
    print("Bangladesh Tax Calculator Agent (2025-2026)")
    print("="*60)
    print("Type 'quit' to exit\n")
    
    conversation = []
    
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Goodbye!")
            break
        
        if not user_input:
            continue
        
        try:
            response, conversation = chat(user_input, conversation)
            print(f"\nAgent: {response}\n")
        except Exception as e:
            print(f"Error: {e}\n")

#Uncomment to run:
interactive_chat()